In [2]:
import json

car_group = dict()


k5 = dict()

k5["price"] = "5000"

k5["year"] = "2015"

car_group["K5"] = k5


avante = dict()

avante["price"] = "3000"

avante["year"] = "2014"

car_group["Avante"] = avante


#json 파일로 저장

with open('./test.json', 'w', encoding='utf-8') as make_file:

    json.dump(car_group, make_file, indent="\t")


# 저장한 파일 출력하기

with open('./test.json', 'r') as f:

    json_data = json.load(f)

print(json.dumps(json_data, indent="\t"))


{
	"K5": {
		"price": "5000",
		"year": "2015"
	},
	"Avante": {
		"price": "3000",
		"year": "2014"
	}
}


In [3]:
import fitz
from operator import itemgetter


In [ ]:
import json

doc = dict()

page = dict()
bbox = dict()
text = dict()

